In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from AttentionModule import Conv2d_Attn

import torch
from torch import nn
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import re
import numpy as np

In [3]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [4]:
resnet_pretrained = models.resnet50(pretrained=True)
nn.Conv2d = Conv2d_Attn
resnet_attn = models.resnet50()
resnet_attn.load_state_dict(resnet_pretrained.state_dict(), strict=False)

In [5]:
# This block turns 'layer1.0.downsample.0.weight' to 'layer1[0].downsample[0].weight'
param_keys = list(resnet_attn.state_dict().keys())
formatted_keys = []
for k in param_keys:
    found = re.findall(r'\.[\d]{1,2}\.', k)
    if len(found):
        for f in found:
            k = k.replace(f, '[{}].'.format(f.strip('.')))
    formatted_keys.append(k)

In [6]:
# This block turn off gradient up for all params except attn_weights
def turn_off_grad_except(lst=[]):
    for k in formatted_keys:
        obj = eval('resnet_attn.'+k)
        for kw in lst:
            if not kw in k:
                obj.requires_grad = False
            else:
                obj.requires_grad = True

In [7]:
resnet_attn.fc = nn.Linear(resnet_attn.fc.in_features, 144)

Start training

In [8]:
batch_size = 32
# batch_size = 64

In [9]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

trainset = torchvision.datasets.ImageFolder(root='../data/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valset = torchvision.datasets.ImageFolder(root='../data/val', transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [10]:
total_imgs = len(trainset.imgs)

In [11]:
resnet_attn = resnet_attn.cuda()

In [12]:
total_attn_params = 0
for k in formatted_keys:
    obj = eval('resnet_attn.'+k)
    if 'attn_weights' in k:
        total_attn_params += np.prod(obj.shape)
print("Total number of attention parameters", total_attn_params)

Total number of attention parameters 26560


We want the attention parameters to diverge from 1, therefore we penalize element-wise square loss as $\lambda (1 \times \text{# params} - (x - 1)^2)$

But this is too big a number,
let's try: 
$- (x - 1)^2$ for now

In [13]:
_lambda = 1e-2 #set default

In [14]:
def get_params_objs(name, net='resnet_attn'):
    res = []
    for k in formatted_keys:
        obj = eval(f'{net}.'+k)
        if name in k:
            res.append(obj)
    return res

In [15]:
def compute_attn_loss(n_params=26560):
    attns = get_params_objs('attn_weights')
    penality = sum([torch.pow(t - 1,2).mean() for t in attns])
    return _lambda*(- penality)

In [16]:
print_every = 50

In [17]:
def score_top3(train=True, val=True):
    if train:
        correct_count = 0
        for inp, label in tqdm(iter(trainloader)):
            _, idx = resnet_attn(Variable(inp).cuda()).topk(3)
            lab = Variable(label).cuda()
            lab_expand = lab.unsqueeze(1).expand_as(idx)
            correct_count += int((idx == lab_expand).sum())
        print(correct_count/len(trainset))
    
    if val:
        correct_count = 0
        for inp, label in tqdm(iter(valloader)):
            _, idx = resnet_attn(Variable(inp).cuda()).topk(3)
            lab = Variable(label).cuda()
            lab_expand = lab.unsqueeze(1).expand_as(idx)
            correct_count += int((idx == lab_expand).sum())
        print(correct_count/len(valset))

In [18]:
def plot_attn_hist():
    attns = get_params_objs('attn_weights')
    attns = torch.cat([attn.squeeze() for attn in attns])
    attns_arr = attns.data.cpu().numpy()
    plt.hist(attns_arr)

In [19]:
def train(k=1, add_attn=True, score=True, plot_hist=False):
    cls_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet_attn.parameters()))
    
    for j in range(k):
        running_loss = 0.0
        running_attn_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

            optimizer.zero_grad()
            outputs = resnet_attn(inputs)
            loss = cls_criterion(outputs, labels)
            attn_loss = compute_attn_loss()
            if add_attn:
                loss += attn_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            running_attn_loss += attn_loss.data[0]

            if i % print_every == 0:
                print('[%5d] iter, [%2f] epoch, avg loss: %.3f, attn_loss: %.5f ' %
                      (i + 1, i*batch_size/total_imgs, running_loss/print_every, running_attn_loss/print_every))
                running_loss = 0.0
                running_attn_loss = 0.0
        if score:
            score_top3()
        if plot_hist:
            plot_attn_hist()
            plt.show()

In [20]:
turn_off_grad_except(['fc'])
resnet_attn.eval() # Turn on batchnorm
train(1, add_attn=False, plot_hist=False)

[    1] iter, [0.000000] epoch, avg loss: 0.101, attn_loss: 0.00000 
[   51] iter, [0.196054] epoch, avg loss: 4.239, attn_loss: 0.00000 
[  101] iter, [0.392109] epoch, avg loss: 3.486, attn_loss: 0.00000 
[  151] iter, [0.588163] epoch, avg loss: 3.236, attn_loss: 0.00000 
[  201] iter, [0.784218] epoch, avg loss: 3.051, attn_loss: 0.00000 
[  251] iter, [0.980272] epoch, avg loss: 2.987, attn_loss: 0.00000 


100%|██████████| 256/256 [01:04<00:00,  3.99it/s]

0.5934321774292366



100%|██████████| 55/55 [00:14<00:00,  3.93it/s]

0.5119726339794755


In [21]:
turn_off_grad_except(['attn_weights'])
resnet_attn.eval() # Turn on batchnorm
_lambda=1
train(10, add_attn=False, plot_hist=False)

[    1] iter, [0.000000] epoch, avg loss: 0.047, attn_loss: 0.00000 
[   51] iter, [0.196054] epoch, avg loss: 2.285, attn_loss: -0.00294 
[  101] iter, [0.392109] epoch, avg loss: 2.299, attn_loss: -0.00902 
[  151] iter, [0.588163] epoch, avg loss: 2.287, attn_loss: -0.01449 
[  201] iter, [0.784218] epoch, avg loss: 2.166, attn_loss: -0.02115 
[  251] iter, [0.980272] epoch, avg loss: 2.221, attn_loss: -0.02794 


100%|██████████| 256/256 [01:04<00:00,  3.95it/s]

0.6630314912388188



100%|██████████| 55/55 [00:14<00:00,  3.92it/s]

0.5775370581527937


...

0.5860889395667047


[    1] iter, [0.000000] epoch, avg loss: 0.028, attn_loss: -0.00158 
[   51] iter, [0.196054] epoch, avg loss: 2.011, attn_loss: -0.08489 
[  101] iter, [0.392109] epoch, avg loss: 2.045, attn_loss: -0.09638 
[  151] iter, [0.588163] epoch, avg loss: 1.983, attn_loss: -0.10647 
[  201] iter, [0.784218] epoch, avg loss: 2.004, attn_loss: -0.11811 
[  251] iter, [0.980272] epoch, avg loss: 2.014, attn_loss: -0.12928 


 45%|████▍     | 115/256 [00:54<01:06,  2.12it/s]Process Process-22:
Traceback (most recent call last):
Process Process-21:
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/ubuntu/miniconda3/lib/python3.6/multiprocessing/pro

Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-18600e2e4153>", line 4, in <module>
    train(10, add_attn=False, plot_hist=False)
  File "<ipython-input-19-0f6e34592bc7>", line 31, in train
    score_top3()
  File "<ipython-input-17-41c6e7e1f1d4>", line 6, in score_top3
    lab = Variable(label).cuda()
  File "/home/ubuntu/miniconda3/lib/python3.6/site-packages/torch/autograd/variable.py", line 298, in cuda
    return CudaTransfer.apply(self, device, async)
  File "/home/ubuntu/miniconda3/lib/python3.6/site-packages/torch/autograd/_functions/tensor.py", line 201, in forward
    return i.cuda(async=async)
  File "/home/ubuntu/miniconda3/lib/python3.6/site-packages/torch/_utils.py", line 69, in _cuda
    return new_type(self.size()).copy_(self, async)
KeyboardInterrupt

During handling of the above except

KeyboardInterrupt: 

In [22]:
torch.save(resnet_attn, '58_6_val_attn.pkl')

In [23]:
resnet_attn

ResNet(
  (conv1): Conv2d_Attn(
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d_Attn(
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d_Attn(
      )
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d_Attn(
      )
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d_Attn(
        )
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bottleneck(
      (conv1): Conv2d_Attn(
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d_Attn(
      )
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (co